In [1]:
import pandas as pd

In [2]:
# Read list exported from TradingView
# Support comma or line delimited rows.

from io import StringIO

fn = 'SP500.txt'
s = ','.join([x.rstrip().rstrip(',') for x in open(fn)])
df1 = pd.read_csv(StringIO(s), sep=':', lineterminator=',', names=['Exchange', 'Ticker'])
df1['Exchange'].value_counts()

Exchange
NYSE      349
NASDAQ    153
AMEX        1
Name: count, dtype: int64

In [3]:
# Read list downloaded from State Street.
# https://www.ssga.com/us/en/intermediary/etfs/funds/spdr-sp-500-etf-trust-spy

fn = 'holdings-daily-us-en-spy.xlsx'

# Read data skipping preamble
df2 = pd.read_excel(fn, skiprows=4)

# Drop Dollar row (lacks a symbol) and postamble
r = df2[df2.isnull().all(axis=1) == True].index.tolist()[0]
df2 = df2.loc[0:r-1]
df2 = df2[df2['Name'] != 'US DOLLAR']

In [4]:
removed_tickers = df1[~df1['Ticker'].isin(df2['Ticker'])]['Ticker']
added_tickers = df2[~df2['Ticker'].isin(df1['Ticker'])]['Ticker']

In [5]:
removed_tickers

Series([], Name: Ticker, dtype: object)

In [6]:
added_tickers

Series([], Name: Ticker, dtype: object)

In [7]:
# Build a new list updated with removed and added tickers.
# Use Yahoo Finance to lookup exchange of added tickers.

import yfinance as yf

def ticker_to_exchange(ticker):
    tickerdata = yf.Ticker(ticker.replace('.', '-'))
    d = {'NYQ': 'NYSE',
        'NMS': 'NASDAQ',
        'BTS': 'AMEX'}
    return d[tickerdata.info['exchange']]    

df1b = df1[~df1['Ticker'].isin(removed_tickers)]

for x in added_tickers:
    d = {'Exchange': [ticker_to_exchange(x)], 'Ticker': [x]}
    df1b = pd.concat([df1b, pd.DataFrame(d)])

df1b = df1b.sort_values('Ticker')

In [8]:
# Find change of exchange.
# Sometimes Yahoo gets this wrong. (E.g., exchange code NGM for NASDAQ:ENPH)

lst = []
for index, row in df1b.iterrows():
    exchange0 = row['Exchange']
    ticker = row['Ticker']
    try:
        exchange1 = ticker_to_exchange(ticker)
    except KeyError as e:
        print('Unknown exchange:', e, ticker, exchange0)
    else:
        if exchange0 != exchange1:
            lst += [[ticker, exchange0, exchange1]]

df1c = pd.DataFrame(lst, columns=['Ticker', 'OldExchange', 'NewExchange'])

df1d = df1b[~df1b['Ticker'].isin(df1c['Ticker'])]
for x in df1c['Ticker']:
    d = {'Exchange': [ticker_to_exchange(x)], 'Ticker': [x]}
    df1d = pd.concat([df1d, pd.DataFrame(d)])

df1d = df1d.sort_values('Ticker')

df1c

Unknown exchange: 'NGM' ENPH NASDAQ


,Ticker,OldExchange,NewExchange


In [9]:
# Write TradingView importable list.

open('SP500_new.txt', 'w').write(',\n'.join((df1d['Exchange'] + ':' + df1b['Ticker']).tolist()))

5419